# Request and Download Data for Multiple Instruments
*Written by Leila Belabbassi and Lori Garzio, Rutgers University*

Purpose: demonstrate how to use interactive tools to build netCDF data request urls for pre-selected instruments in a .csv file, and send the requests. Several functions were already built to make the process easier and are included in the *scripts* and *functions* folders. Note: the functions were written to only download data containing scientific parameters (excludes engineering-only data streams).

The .csv file with a pre-selected list of instruments must have the following columns:
['reference_designator', 'delivery_method']. An example file (instrument_list.csv) is included. You can modify this file to download other instruments from other platforms. To find instruments to download, visit the [OOI Data Portal](https://ooinet.oceanobservatories.org).

You must also provide a directory where outputs of the tool are saved. This includes 1) a summary of the status of the data request, the request url, and the outputUrl (THREDDs location where the files are saved), 2) the list of the request urls, 3) the dates for which data were requests and the deployments those dates span, and 4) a list of the urls that were not sent (e.g., if the process was stopped).

### Input variables:

In [1]:
#directory where outputs are saved
sDir = '/Users/lgarzio/Documents/OOI/EGOMeeting/'

username = '' #OOI API username
token = '' # OOI API password

# file with pre-selected list of instruments
f = '/Users/lgarzio/Documents/repo/OOI/ooi-data-lab/8thEGOMeeting-notebooks/instrument_list.csv'

### Import functions:

In [2]:
import datetime as dt
import pandas as pd
import os
import itertools
import functions.common as cf
from scripts import data_request_urls_ooi1_0
from scripts import send_data_requests_nc
from scripts import data_request_tools

### Read input file:

In [3]:
df = pd.read_csv(f)

### Create request urls for instruments in the read file:
Note: the data_request_tools functions *check_str* and *format_inputs* simply makes sure the various components of the reference designator are in the correct format to build the data request url. 

In [4]:
now = dt.datetime.now().strftime('%Y%m%dT%H%M')
url_list = []
for i, j in df.iterrows():
    refdes = j['reference_designator']
    array = refdes[0:2]
    array = data_request_tools.check_str(array)
    array = data_request_tools.format_inputs(array)
    subsite = refdes.split('-')[0]
    subsite = data_request_tools.check_str(subsite)
    subsite = data_request_tools.format_inputs(subsite)
    node = refdes.split('-')[1]
    node = data_request_tools.check_str(node)
    node = data_request_tools.format_inputs(node)
    inst = '-'.join((refdes.split('-')[2], refdes.split('-')[3]))
    inst = data_request_tools.check_str(inst)
    inst = data_request_tools.format_inputs(inst)
    delivery_methods = j['delivery_method']
    delivery_methods = data_request_tools.check_str(delivery_methods)
    delivery_methods = data_request_tools.format_inputs(delivery_methods)
    print(refdes, delivery_methods)
    urls = data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)
    url_list.append(urls)
    print(url_list)

CP05MOAS-GL336-03-CTDGVM000 ['telemetered']
[['https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/03-CTDGVM000/telemetered/ctdgv_m_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true']]
CP05MOAS-GL336-04-DOSTAM000 ['telemetered']
[['https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/03-CTDGVM000/telemetered/ctdgv_m_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true'], ['https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/04-DOSTAM000/telemetered/dosta_abcdjm_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true']]
CP05MOAS-GL336-05-PARADM000 ['telemetered']
[['https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/03-CTDGVM000/telemetered/ctdgv_m_glider

### Send url requests:

The *send_data_requests_nc* main function will send the data request urls that were built in the previous step using the provided credentials as authentication. The function will inform the user how many data requests there are and will ask the user to confirm that the requests should be sent.

In [5]:
f_url_list = list(itertools.chain(*url_list))

thredds_output_urls = send_data_requests_nc.main(sDir, f_url_list, username, token, now)


There are 8 requests to send, are you sure you want to continue? y/<n>: y

Request url 1 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/03-CTDGVM000/telemetered/ctdgv_m_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data available for request

Request url 2 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/04-DOSTAM000/telemetered/dosta_abcdjm_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data available for request

Request url 3 of 8: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP05MOAS/GL336/05-PARADM000/telemetered/parad_m_glider_instrument?beginDT=2015-01-11T20:57:00.000Z&endDT=2018-04-03T11:50:00.000Z&include_annotations=true&include_provenance=true
Data request sent
Data available for requ

### Seeing if the requests have fulfilled:

This function checks the THREDDs location where the files are saved to determine if the request has fulfilled.

In [6]:
for i in range(len(thredds_output_urls)):
    url = thredds_output_urls[i]
    print('\nDataset {} of {}: {}'.format((i + 1), len(thredds_output_urls), url))
    if 'no_output_url' not in url:
        cf.check_request_status(url)


Dataset 1 of 8: https://opendap.oceanobservatories.org/thredds/catalog/ooi/lgarzio@marine.rutgers.edu/20190520T140340-CP05MOAS-GL336-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument/catalog.html
Data request is still fulfilling. Trying again in 1 minute.
Data request has fulfilled.

Dataset 2 of 8: https://opendap.oceanobservatories.org/thredds/catalog/ooi/lgarzio@marine.rutgers.edu/20190520T140340-CP05MOAS-GL336-04-DOSTAM000-telemetered-dosta_abcdjm_glider_instrument/catalog.html
Data request has fulfilled.

Dataset 3 of 8: https://opendap.oceanobservatories.org/thredds/catalog/ooi/lgarzio@marine.rutgers.edu/20190520T140340-CP05MOAS-GL336-05-PARADM000-telemetered-parad_m_glider_instrument/catalog.html
Data request has fulfilled.

Dataset 4 of 8: https://opendap.oceanobservatories.org/thredds/catalog/ooi/lgarzio@marine.rutgers.edu/20190520T140340-CP05MOAS-GL336-02-FLORTM000-telemetered-flort_m_sample/catalog.html
Data request is still fulfilling. Trying again in 1 minute.
Data reque

If you are interested in more ways to download data, including more interactive scripts, check out our data-download repo here: https://github.com/ooi-data-lab/data-download